In [2]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 23.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12

In [3]:
import datasets
import transformers
from transformers import AutoModelForMaskedLM, AutoTokenizer

In [4]:
lang = ['kin'] # only languages related to rw

In [37]:
af_ds_list = []

for l in lang:
    try:
        ds = datasets.load_dataset('shmuhammad/AfriSenti-twitter-sentiment', l)
        af_ds_list.append(ds)
    except:
        print(f'error at {l}')

In [38]:
for i, ds in enumerate(af_ds_list):
    try:
        first_record = ds['train'][0]  # Assuming 'train' split exists
        print(f"First record of dataset {lang[i]}: {first_record}")
        break
    except Exception as e:
        print(f'Error accessing the first record of dataset {lang[i]}: {e}')

First record of dataset kin: {'tweet': '@user @user @user @user @user @user @user Hhhhhh ntabyihogoza, ubu x abo yishe bangana ik', 'label': 2}


In [39]:
for i in range(len(af_ds_list)):
    print(f'{lang[i]}')
    print(af_ds_list[i]['train'][0])

kin
{'tweet': '@user @user @user @user @user @user @user Hhhhhh ntabyihogoza, ubu x abo yishe bangana ik', 'label': 2}


In [40]:
count = 0

for ds in af_ds_list:
    for split in ['train', 'test', 'validation']:
        if split in ds:
            count += ds[split].num_rows

count

5155

In [41]:
full_text = []

for dataset in af_ds_list:
    for split in ['train', 'test', 'validation']:
        if split in dataset:
            texts = dataset[split]['tweet']
            full_text.extend(texts)


len(full_text)

5155

In [42]:
full_text[:5]

['@user @user @user @user @user @user @user Hhhhhh ntabyihogoza, ubu x abo yishe bangana ik',
 '@user Amahano?! Ni impanuka, inkangu, inzara.... Muyite izina rikwiye.',
 'Ese umuntu aguhaye miliyoni 7 zidorali ngo aryamane numugore wawe cg umukunzi wawe wabyemera🙄????',
 'Ugira amagambo😏 kandi Ubwo wasanga nawe byagutabara. Jya uvuga uziga😏 https://t.co/9OWOZxquFO',
 'Ukuntu inama zose zikomeye zirikubera Mu Rwanda ubanza numusi wimperuka uzabera kigali 🤔']

In [24]:
!pip install demoji

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 1.9 MB/s eta 0:00:00


In [43]:
import re
import demoji

In [44]:
def clean_text(text):

    text = re.sub(r'http\S+|www.\S+|@\w+', '', text)
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    text = text.strip()
    text = demoji.replace(text, "")

    return text

In [48]:
full_text = pd.DataFrame(data=full_text, columns=['text'])
full_text = full_text['text'].apply(clean_text)

In [57]:
full_text.head()

0       Hhhhhh ntabyihogoza ubu x abo yishe bangana ik
1    Amahano Ni impanuka inkangu inzara Muyite izin...
2    Ese umuntu aguhaye miliyoni 7 zidorali ngo ary...
3    Ugira amagambo kandi Ubwo wasanga nawe byaguta...
4    Ukuntu inama zose zikomeye zirikubera Mu Rwand...
Name: text, dtype: object

In [50]:
import pandas as pd

# kirnews_train = pd.read_csv('news-datasets/KIRNEWS/cleaned/train.csv')
# kirnews_test = pd.read_csv('news-datasets/KIRNEWS/cleaned/test.csv')

kinnews_train = pd.read_csv('/content/drive/MyDrive/Research/Uni of Pretoria /KINNEWS/Copy of train.csv')
kinnews_test = pd.read_csv('/content/drive/MyDrive/Research/Uni of Pretoria /KINNEWS/Copy of test.csv')

In [51]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [52]:
len(kinnews_train), len(kinnews_test)

(17014, 4254)

In [53]:
news = pd.concat([kinnews_train, kinnews_test])
news

,label,title,content
0,3,bugesera nyuma y’inzara yaharanzwe karahahira ...,mu myaka yashize cyane cyane umwaka w’ akarere...
1,5,mutzig beer fest itegerejwe n’abantu benshi ki...,mutzig beer fest thedition izabera juru parki ...
2,2,abakinnyi bamagare batanu berekeje isiganwa ny...,abakinnyi batanu berekeje asmara adrien niyons...
3,3,kireheumugabo afungiwe gufatanwa imiti irimo c...,inyubako nshyashya new kirehe administrative b...
4,7,sobanukirwa internet g izahindura ubuzima muntu,ni umuyoboro ugereranywa nk’intambwe nshya nde...
...,...,...,...
4249,2,volleyball apr bagabo rra bagore zegukanye car...,nyuma kubura igikombe shampiyona umwaka ushize...
4250,1,gisagara imbwa zizerera zikomeje kwibasira ama...,ibi abaturage babitangaje nyuma yaho mbwa zima...
4251,3,rdb yahembye batanu bagaragaje gutanga service...,ikigo k’igihugu gishinzwe iterambere rdb cyash...
4252,2,reba ibyamamare bizwi cyane byanze ’kurushinga...,gushyingirwa ntabwo garanti y’ibyishimo bamwe ...


In [58]:
full_text = full_text.tolist()

In [60]:
full_text[:5]

['Hhhhhh ntabyihogoza ubu x abo yishe bangana ik',
 'Amahano Ni impanuka inkangu inzara Muyite izina rikwiye',
 'Ese umuntu aguhaye miliyoni 7 zidorali ngo aryamane numugore wawe cg umukunzi wawe wabyemera',
 'Ugira amagambo kandi Ubwo wasanga nawe byagutabara Jya uvuga uziga',
 'Ukuntu inama zose zikomeye zirikubera Mu Rwanda ubanza numusi wimperuka uzabera kigali']

In [61]:
for index, row in news.iterrows():
    title = row['title']
    content = row['content']
    full_text.append(title)
    full_text.append(content)

len(full_text)

47691

In [65]:
full_text[-3:]

['gushyingirwa ntabwo garanti y’ibyishimo bamwe bashakanye ubuzima bwarahindutse buba bwiza gihe abandi bwababereye ukundi abadashoboye kwihanganira bagahitamo gutandukana bamwe byamamare bazwi mumateka bashobora kuba batari abakunzi b’ibyo gushyingirwa cyangwa se bahisemo kuguma ingaragu kubera izindi mpamvu ibyo byose bafite ishingiro kuko tutareka kuvuga ibintu byabageze neza urebye bagezeho buzima bwabo oprah winfrey mu oprah winfrey yahuye mugenzi w’ubuzima yambikwa impeta y’urukundo ariko nyuma ahitamo kubivamo arigendera yagaragaje yahisemo kuguma umuseribateri kandi adashaka kubyaramu kiganiro oprah winfrey yagiranye n’abantu yabasobanuriye afite ubundi buryo kwerekana ububyeyi bwe bikaba byaratumye ashinga ishuri rikuru ry’ubuyobozi ry’abakobwa afurika y’epfo ryitwa oprah winfrey leadership academy for girls’ ati abo bakobwa buzuza nshingano kibyeyi wenda nagombaga kugira mubyukuri baruzura-nuzuyemo umubyeyi condoleezza rice nk’uwahoze umunyamabanga leta condoleezza rice umwe 

In [79]:
with open('/content/drive/MyDrive/Research/Uni of Pretoria/text_dump.txt', 'w') as f:
    for text in full_text:
        f.write(str(text)+'\n')

In [68]:
from tokenizers import ByteLevelBPETokenizer
from transformers import XLMRobertaTokenizerFast
from datasets import load_dataset
import os

def get_training_corpus(dataset, batch_size=1000):
    for start_idx in range(0, len(dataset), batch_size):
        samples = dataset[start_idx : start_idx + batch_size]
        yield samples["text"]

dataset = load_dataset("text", data_files={"train": "text_dump.txt"})["train"]

dataset

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['text'],
    num_rows: 47691
})

In [ ]:
# trying with sentencepiece from transformers

In [69]:
from tokenizers import SentencePieceBPETokenizer

tokenizer = SentencePieceBPETokenizer()

tokenizer.train_from_iterator(
    get_training_corpus(dataset),
    vocab_size=250000,
    min_frequency=2,
    show_progress=True,
)

In [ ]:
tokenizer

Tokenizer(vocabulary_size=229491, model=SentencePieceBPE, unk_token=<unk>, replacement=▁, add_prefix_space=True, dropout=None)

In [70]:
tokenizer

Tokenizer(vocabulary_size=162267, model=SentencePieceBPE, unk_token=<unk>, replacement=▁, add_prefix_space=True, dropout=None)

In [71]:
xlmr_tokenizer = XLMRobertaTokenizerFast(
    tokenizer_object=tokenizer
)

In [72]:
xlmr_tokenizer

XLMRobertaTokenizerFast(name_or_path='', vocab_size=162267, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	162267: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	162268: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	162269: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	162270: AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False, special=True),
}

In [77]:
os.makedirs('/content/drive/MyDrive/Research/Uni of Pretoria/xlmr-tokenizer', exist_ok=True)
xlmr_tokenizer.save_pretrained('xlmr-tokenizer')

('xlmr-tokenizer/tokenizer_config.json',
 'xlmr-tokenizer/special_tokens_map.json',
 'xlmr-tokenizer/tokenizer.json')

In [75]:
from transformers import ElectraTokenizerFast

electra_tokenizer = ElectraTokenizerFast(
    tokenizer_object=tokenizer
)

electra_tokenizer

ElectraTokenizerFast(name_or_path='', vocab_size=162267, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	162267: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	162268: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	162269: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	162270: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	162271: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=

In [78]:
electra_tokenizer.save_pretrained('/content/drive/MyDrive/Research/Uni of Pretoria/electra-tokenizer')

('/content/drive/MyDrive/Research/Uni of Pretoria/electra-tokenizer/tokenizer_config.json',
 '/content/drive/MyDrive/Research/Uni of Pretoria/electra-tokenizer/special_tokens_map.json',
 '/content/drive/MyDrive/Research/Uni of Pretoria/electra-tokenizer/vocab.json',
 '/content/drive/MyDrive/Research/Uni of Pretoria/electra-tokenizer/merges.txt',
 '/content/drive/MyDrive/Research/Uni of Pretoria/electra-tokenizer/added_tokens.json',
 '/content/drive/MyDrive/Research/Uni of Pretoria/electra-tokenizer/tokenizer.json')